In [ ]:
%store -r model_data
%store -r s3_bucket
%store -r prefix

In [ ]:
input_data = "s3://{}/{}/data/input.json".format(s3_bucket,prefix)
input_data

In [ ]:
!aws s3 cp data/input.json {input_data}

In [ ]:
from sagemaker import get_execution_role 

role = get_execution_role()

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig

output_path = f"s3://{s3_bucket}/{prefix}/output"
async_config = AsyncInferenceConfig(output_path=output_path)

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    model_data=model_data, 
    role=role, 
    source_dir="scripts",
    entry_point='inference.py', 
    framework_version='1.6.0',
    py_version="py3"
)

In [ ]:
%%time

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    instance_type='ml.m5.xlarge', 
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    async_inference_config=async_config
)

In [ ]:
response = predictor.predict_async(input_path=input_data)

In [ ]:
from time import sleep

sleep(40)

response.get_result()

In [ ]:
output_path = response.output_path
output_path

In [ ]:
!aws s3 cp {output_path} sample.out

In [ ]:
!cat sample.out

In [ ]:
!rm sample.out

In [ ]:
predictor.delete_endpoint()